In [2]:
import json

from snowflake.snowpark import Session
from snowflake.snowpark.functions import col as F_col, max as F_max

from snowflake.core import Root
from snowflake.core.task import Task
from snowflake.core.task import StoredProcedureCall
from snowflake.core.task import Cron

In [5]:
# Create session connection
session = Session.builder.configs(json.load(open("connection.json"))).create()

# Make Data Engineering Function

In [6]:
def data_engineering(session: Session) -> str:
    """
    Perform data engineering tasks on raw data from Snowflake
    and save the transformed data as a new table.

    This function is designed to prepare data for further analysis or 
    modeling by transforming raw input into a structured format. It 
    ensures that all necessary preprocessing steps are completed, such 
    as handling missing values and normalizing category proportions. 

    Args:
        session (Session): An active Snowflake session object
                          used to run SQL operations.

    Returns:
        str: A status message indicating the completion of the
             data engineering process.
    """

    # Get raw data
    raw_data_a = session.table("raw_data_a")

    # Select and aggregate
    model_one_df = (
        raw_data_a.select(
            "id",
            "lot",
            "site",
            "storage",
            "start_date",
            "end_date",
            "type",
            "category_1",
            "category_2",
            "category_3",
        )
        .filter(F_col("lot").is_not_null())
        .group_by(
            "id",
            "site",
            "storage",
            "start_date",
            "type",
            "category_1",
            "category_2",
            "category_3",
        )
        .agg(F_max("end_date").alias("max_end_date"))
        .distinct()
        .order_by("id")
    )

    # Fill empty category measures
    model_one_df = model_one_df.fillna(
        0,
        subset=["category_1", "category_2", "category_3"]
    )

    # Add calculated total F_column
    model_one_df = model_one_df.withF_column(
        "total",
        F_col("category_1") + F_col("category_2") + F_col("category_3")
    )

    # Filter out rows with no total
    model_one_df = model_one_df.filter(F_col("total") != 0)

    # Calculate percentage of totals for category 1
    model_one_df = model_one_df.withF_column(
        "category_1_pct", F_col("category_1") / F_col("total")
    )

    # Calculate percentage of totals for category 2
    model_one_df = model_one_df.withF_column(
        "category_2_pct", F_col("category_2") / F_col("total")
    )

    # Calculate percentage of totals for category 3
    model_one_df = model_one_df.withF_column(
        "category_3_pct", F_col("category_3") / F_col("total")
    )

    # Write table to Snowflake
    model_one_df.write.mode("overwrite").save_as_table("data_model_one")

    return "success"

In [ ]:
# Register the data_engineering function as a stored procedure in Snowflake
procedure = session.sproc.register(
    func=data_engineering,
    name="data_engineering",
    packages=["snowflake-snowpark-python"],
    is_permanent=True,
    stage_location="@ML",
    replace=True
)

# Create data engineering task with SQL

In [ ]:
# SQL to create the task
create_task_sql = """
CREATE OR REPLACE TASK data_engineering_task
WAREHOUSE = fishtalk_ml_warehouse 
SCHEDULE = "USING CRON 0 0 * * * UTC"
COMMENT = "Daily run of data_engineering"
AS
CALL data_engineering();
"""

# SQL to enable the task
enable_task_sql = "ALTER TASK data_engineering_task RESUME"

# Create the task
session.sql(create_task_sql).collect()

# Enable the task
session.sql(enable_task_sql).collect()

# Create data engineering task with Python


In [13]:
# Create the task object
data_engineering_task_py = Task(
    definition=StoredProcedureCall(procedure),
    name="data_engineering_task_py",
    warehouse="fishtalk_ml_warehouse",
    schedule=Cron("0 0 * * *", "UTC"),
    comment="Daily run of data_engineering",
)

# Create the task
root = Root(session)
tasks = root.databases["data"].schemas["ml"].tasks
tasks.create(data_engineering_task_py)

# Enable the task
task_res = tasks["data_engineering_task_py"]
task_res.execute()